In [18]:
import h5py
import numpy as np
from skimage import io, transform
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import sys,os
sys.path.append(os.path.abspath('../'))
import utils.Dataset as u
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as fc
import torch.nn.modules.normalization as nm
import torch.optim as optim
import CNN as cnn

In [5]:
a= 5
torch.tensor(a)

tensor(5)

In [2]:
directory = "/media/3522046/CHEKIROU/SoliData/dsp/"
# Datasets
train, test = u.split(directory,frames = True, percentage = 0.8, use= 0.01) # returns a split of the frames 80% train, 20% test, for 80% of the data

t = transforms.Compose([u.Reshape(), u.Rescale((224,224)), u.ToTensor()])
training_set = u.Data(directory,train,transform = t) # open the dataset

In [4]:
len(test)

656

In [3]:
training_set[0][1]

/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


tensor([9], dtype=torch.int32)

In [20]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True # ca marche pas chez moi,peut etre qu'il faut que j'installe un truc
# Parameters, num_workers c'est les threads
"""device = torch.device("cpu")"""
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 6}

max_epochs = 1 # j'ai mis a 1 pour le test pour aller vite
directory = "/tmp/SoliData/dsp"
# Datasets
train, test = u.split(directory,frames = True, percentage = 0.8, use= 0.1) # returns a split of the frames 80% train, 20% test, for 80% of the data

# Generators

t = transforms.Compose([u.Reshape(), u.Rescale((224,224)), u.ToTensor()]) # composition of transformations
training_set = u.Data(directory,train,transform = t) # open the dataset
training_generator = DataLoader(training_set, **params)


validation_set = u.Data(directory,test,transform = t) # open the dataset
validation_generator = DataLoader(validation_set, **{'batch_size': 1,'shuffle': False,'num_workers': 6})

model = cnn.Net().double().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
# Train the model


loss_list = []
acc_list = []
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, labels) in enumerate(training_generator):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        labels = labels.view(images.shape[0]).long()
        
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        del images
        del labels
        if i % 100  == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        

/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be 

[1,   100] loss: 0.124
[1,   200] loss: 0.124
[1,   300] loss: 0.124
[1,   400] loss: 0.124


In [32]:
training_set2 = u.Data(directory,train[:200],transform = t)

In [33]:
training_generator2 = DataLoader(training_set2, **{'batch_size': 1,'shuffle': False,'num_workers': 6})

In [36]:
acc = 0
for images, labels in (training_generator2):
    images, labels = images.to(device), labels.long().to(device)
    outputs = model(images)
    print(outputs)
    print("------------------")
    prob_dist = torch.distributions.Categorical(outputs) # probs should be of size batch x classes
    predictions = prob_dist.sample().to(device)
    acc += int(predictions == labels[0])
    
accuracy = acc / len(training_generator2)

/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect'

tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0884, 0.0784, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0807, 0.0935, 0.0907, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0884, 0.0785, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0883, 0.0784, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0807, 0.0935, 0.0908, 0.0803, 0.0840, 0.0806, 0.0841, 0.0803, 0.0890,
         0.0883, 0.0785, 0.0700]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0884, 0.0784, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<So

tensor([[0.0807, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0883, 0.0785, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0934, 0.0908, 0.0803, 0.0840, 0.0806, 0.0841, 0.0804, 0.0889,
         0.0884, 0.0784, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0841, 0.0805, 0.0841, 0.0803, 0.0890,
         0.0883, 0.0784, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0883, 0.0785, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0890,
         0.0884, 0.0785, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<So

tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0889,
         0.0883, 0.0784, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------
tensor([[0.0806, 0.0935, 0.0908, 0.0803, 0.0840, 0.0805, 0.0841, 0.0803, 0.0890,
         0.0883, 0.0785, 0.0701]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
------------------


KeyboardInterrupt: 

In [35]:
accuracy

0.08

In [16]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [6]:
torch.cuda.is_available()

True

In [19]:
torch.cuda.empty_cache()

In [8]:
probs = torch.Tensor([ [0.1, 0.2, 0.7], [0.1, 0.8, 0.1], [0.4, 0.5, 0.1] ])
prob_dist = torch.distributions.Categorical(probs) # probs should be of size batch x classes
a = prob_dist.sample()

In [9]:
a

tensor([1, 1, 0])